In [2]:

from pathlib import Path
import re

from bs4 import BeautifulSoup

bangumi_html = Path("bangumi.txt").read_text(encoding="utf-8")
soup = BeautifulSoup(bangumi_html, "html.parser")

id_name_mapping = {}
for subtitle in soup.select("div.item h2.subtitle"):
    anchor = subtitle.find("a", href=re.compile(r"^/character/\d+"))
    if not anchor:
        continue

    match = re.search(r"/character/(\d+)", anchor["href"])
    if not match:
        continue

    character_id = match.group(1)
    japanese_name = anchor.get_text(strip=True)

    chinese_span = subtitle.find("span", class_="tip")
    chinese_name = chinese_span.get_text(strip=True) if chinese_span else ""

    id_name_mapping[character_id] = {
        "ja": japanese_name,
        "zh": chinese_name,
    }

for character_id, names in id_name_mapping.items():
    zh_name = f" / {names['zh']}" if names["zh"] else ""
    print(f"{character_id}: {names['ja']}{zh_name}")


187565: レッドディザイア / 红色梦想
50576: スペシャルウィーク / 特别周
50577: サイレンススズカ / 无声铃鹿
50578: トウカイテイオー / 东海帝王
50599: マルゼンスキー / 丸善斯基
50586: オグリキャップ / 小栗帽
50593: タイキシャトル / 大树快车
50600: メジロマックイーン / 目白麦昆
50592: シンボリルドルフ / 鲁道夫象征
68386: ライスシャワー / 米浴
50588: ゴールドシップ / 黄金船
50580: ウオッカ / 伏特加
50594: ダイワスカーレット / 大和赤骥
50587: グラスワンダー / 草上飞
50585: エルコンドルパサー / 神鹰
50584: エアグルーヴ / 气槽
68392: マヤノトップガン / 重炮
60561: スーパークリーク / 超级溪流
60529: メジロライアン / 目白赖恩
68383: アグネスタキオン / 爱丽速子
60522: ウイニングチケット / 胜利奖券
62513: サクラバクシンオー / 樱花进王
60527: ハルウララ / 春乌菈菈
60530: マチカネフクキタル / 待兼福来
68394: ナイスネイチャ / 优秀素质
68395: キングヘイロー / 帝王光辉
50595: テイエムオペラオー / 好歌剧
62509: ミホノブルボン / 美浦波旁
62512: ビワハヤヒデ / 琵琶晨光
68388: カレンチャン / 真机伶
60558: ナリタタイシン / 成田大进
60557: スマートファルコン / 醒目飞鹰
50596: ナリタブライアン / 成田白仁
23243: セイウンスカイ / 青云天空
50597: ヒシアマゾン / 菱亚马逊
50598: フジキセキ / 富士奇石
60560: ゴールドシチー / 黄金城市
68385: メイショウドトウ / 名将怒涛
68390: エイシンフラッシュ / 荣进闪耀
62506: ヒシアケボノ / 菱曙
68389: アグネスデジタル / 爱丽数码
60528: カワカミプリンセス / 川上公主
60525: マンハッタンカフェ / 曼城茶座
60526: トーセンジョーダン / 东瀛佐敦
68393: メジロドーベル / 目白多伯
6

In [6]:
from collections import defaultdict

bwiki_html = Path("bwiki.txt").read_text(encoding="utf-8")
bwiki_soup = BeautifulSoup(bwiki_html, "html.parser")

columns = [
    "稀有度",
    "草地",
    "泥地",
    "短距离",
    "英里",
    "中距离",
    "长距离",
    "领跑",
    "跟前",
    "居中",
    "后追",
]

ja_to_ids = defaultdict(list)
for character_id, names in id_name_mapping.items():
    ja_to_ids[names["ja"]].append(character_id)

def make_stat_dict():
    return {column: [] for column in columns}

character_stats = defaultdict(make_stat_dict)

grade_letter_pattern = re.compile(r"[SABCDEFUG]")


def extract_grade(cell):
    hidden = cell.find("div")
    if hidden:
        hidden_text = hidden.get_text(strip=True)
        if hidden_text:
            return hidden_text

    for img in cell.find_all("img", alt=True):
        match = grade_letter_pattern.search(img["alt"])
        if match:
            return match.group(0)

    text = cell.get_text(separator=" ", strip=True)
    match = grade_letter_pattern.search(text)
    if match:
        return match.group(0)

    return None


def extract_initial_stars(cell):
    star_imgs = cell.find_all("img")
    if star_imgs:
        return len(star_imgs)

    digits = re.findall(r"\d+", cell.get_text())
    return int(digits[0]) if digits else None


column_indices = {
    "稀有度": 2,
    "草地": 8,
    "泥地": 9,
    "短距离": 10,
    "英里": 11,
    "中距离": 12,
    "长距离": 13,
    "领跑": 14,
    "跟前": 15,
    "居中": 16,
    "后追": 17,
}

for row in bwiki_soup.select("table#CardSelectTr tbody tr"):
    cells = row.find_all("td")
    if len(cells) <= max(column_indices.values()):
        continue

    name_cell = cells[1]
    ja_name = None
    for span in name_cell.find_all("span", attrs={"lang": "ja"}):
        text = span.get_text(strip=True)
        if not text:
            continue
        if text.startswith("【") and text.endswith("】"):
            continue
        ja_name = text
        break

    if not ja_name:
        anchors = name_cell.find_all("a")
        for anchor in anchors:
            text = anchor.get_text(strip=True)
            if text:
                ja_name = text
                break

    if not ja_name:
        continue

    target_ids = ja_to_ids.get(ja_name)
    if not target_ids:
        continue

    for column, index in column_indices.items():
        value = (
            extract_initial_stars(cells[index])
            if column == "稀有度"
            else extract_grade(cells[index])
        )
        if value is None or value == "":
            continue

        for character_id in target_ids:
            current_values = character_stats[character_id][column]
            if value not in current_values:
                current_values.append(value)

for character_id in sorted(character_stats.keys(), key=int):
    names = id_name_mapping[character_id]
    print(f"{character_id}: {names['ja']} / {names['zh']}")
    for column in columns:
        values = character_stats[character_id][column]
        if values:
            print(f"  {column}: {values}")



23243: セイウンスカイ / 青云天空
  稀有度: [3]
  草地: ['A']
  泥地: ['G']
  短距离: ['G']
  英里: ['C']
  中距离: ['A']
  长距离: ['A']
  领跑: ['A']
  跟前: ['B']
  居中: ['D']
  后追: ['E']
50576: スペシャルウィーク / 特别周
  稀有度: [3]
  草地: ['A']
  泥地: ['G']
  短距离: ['F']
  英里: ['C']
  中距离: ['A']
  长距离: ['A']
  领跑: ['G']
  跟前: ['A']
  居中: ['A']
  后追: ['C']
50577: サイレンススズカ / 无声铃鹿
  稀有度: [3]
  草地: ['A']
  泥地: ['G']
  短距离: ['D']
  英里: ['A']
  中距离: ['A']
  长距离: ['E']
  领跑: ['A']
  跟前: ['C']
  居中: ['E']
  后追: ['G']
50578: トウカイテイオー / 东海帝王
  稀有度: [3]
  草地: ['A']
  泥地: ['G']
  短距离: ['F']
  英里: ['E']
  中距离: ['A']
  长距离: ['B', 'A']
  领跑: ['D']
  跟前: ['A']
  居中: ['C']
  后追: ['E']
50580: ウオッカ / 伏特加
  稀有度: [2, 3]
  草地: ['A']
  泥地: ['G']
  短距离: ['F']
  英里: ['A']
  中距离: ['A']
  长距离: ['F', 'E']
  领跑: ['C']
  跟前: ['B']
  居中: ['A']
  后追: ['F']
50584: エアグルーヴ / 气槽
  稀有度: [2, 3]
  草地: ['A']
  泥地: ['G']
  短距离: ['C']
  英里: ['B']
  中距离: ['A']
  长距离: ['E']
  领跑: ['D']
  跟前: ['A']
  居中: ['A']
  后追: ['G']
50585: エルコンドルパサー / 神鹰
  稀有度: [2, 3]
  草地: ['A']
  泥地

In [9]:
import json

output_path = Path("../../outputs/extra_tags/175552.json")

terrain_columns = ["草地", "泥地"]
distance_columns = ["短距离", "英里", "中距离", "长距离"]
running_style_columns = ["领跑", "跟前", "居中", "后追"]

def should_add_tag(values):
    return any(value in {"A", "B"} for value in values)

tags = {}
for character_id in sorted(character_stats.keys(), key=int):
    stats = character_stats[character_id]
    entry = {}
    entry["_name"] = id_name_mapping[character_id]["zh"]

    rarity_values = sorted({value for value in stats["稀有度"] if isinstance(value, int)})
    if rarity_values:
        entry["稀有度"] = {
            f"{rarity}星": f"<img src='/assets/tag/umamusume/{rarity}star.png' alt='{rarity}星' />"
            for rarity in rarity_values
        }

    terrain_tags = {
        column: column
        for column in terrain_columns
        if should_add_tag(stats[column])
    }
    if terrain_tags:
        entry["场地"] = terrain_tags

    distance_tags = {
        column: column
        for column in distance_columns
        if should_add_tag(stats[column])
    }
    if distance_tags:
        entry["距离"] = distance_tags

    running_tags = {
        column: column
        for column in running_style_columns
        if should_add_tag(stats[column])
    }
    if running_tags:
        entry["跑法"] = running_tags

    if entry:
        tags[character_id] = entry

output_path.write_text(json.dumps(tags, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Wrote {len(tags)} entries to {output_path}")



Wrote 121 entries to 175552.json
